In [5]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [6]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [7]:
formula = """
    ClaimCount ~ 
    TE(Region) + 
    BonusMalus + 
    I(BonusMalus ** 2) + 
    ms(VehAge, df=4) + 
    pos(DrivAge) + 
    pos(I(DrivAge ** 2)) + 
    TE(VehBrand) + 
    TE(Area), 
"""

result = rs.glm(
    formula,
    train_data, 
    family="poisson", 
    offset="Exposure").fit(
)

print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        406439
Link Function:       (default)       Df Residuals:            406427
Method:              IRLS            Df Model:                    11
Scale:               1.0000          Iterations:                  25

Log-Likelihood:         -118837.9676 Deviance:                196128.2855
AIC:                     237699.9352 Null Deviance:           132328.3090
BIC:                     237830.9175 Pearson chi2:              390442.38
Converged:           False          

------------------------------------------------------------------------------
Variable                   Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept               -4.2916     0.0746  -57.526  <0.0001   [ -4.4378,  -4.1454]  ***
BonusMalus               0.0320     0.0013   25

In [8]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': '\n    ClaimCount ~ \n    TE(Region) + \n    BonusMalus + \n    I(BonusMalus ** 2) + \n    ms(VehAge, df=4) + \n    pos(DrivAge) + \n    pos(I(DrivAge ** 2)) + \n    TE(VehBrand) + \n    TE(Area), \n', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 12, 'df_resid': 406427, 'converged': False, 'iterations': 25}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=85056.95, loss=0.508077, deviance=206502.31, log_likelihood=-124024.98, aic=248073.96, gini=0.1597, auc=0.5799, ae_ratio=0.2517, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1718.44, actual=0.605782, predicted=0.262318, ae_ratio=2.3093), DecileMetrics(decile=2, n=40643, exposure=3750.74, actual=0.30314, predicted=0.298572, ae_ratio=1.0153), DecileMetrics(decile=3, n=40643, exposure=7609.41, actual=0.182537, predicted=0.285928, ae_ratio=0.6384), DecileMetrics(dec

In [9]:
result.predict(train_data)

array([0.04535699, 0.47292428, 0.01271514, ..., 0.01360539, 0.21669988,
       0.18785975], shape=(406439,))